In [1]:
import pandas as pd
import numpy as np
import os
import pydicom
import matplotlib.pyplot as plt

from scipy.misc import imsave
from imageio import imwrite, imread
from tqdm import tqdm
from skimage import *
import itertools
import math

%matplotlib inline


import keras
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.preprocessing import image
from keras.applications import xception
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.layers import BatchNormalization
from keras.optimizers import SGD, Adam,Adagrad
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, LearningRateScheduler
from keras.layers.advanced_activations import LeakyReLU
from math import sqrt
from keras.callbacks import History 
from keras.optimizers import Adam, SGD
from keras.layers import Activation
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras.layers.advanced_activations import LeakyReLU
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.preprocessing import image
from keras.applications import xception
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D,GlobalAvgPool2D, GlobalMaxPool2D, GlobalMaxPooling2D, GlobalAveragePooling2D
from keras.layers import Activation, Dropout, Flatten, Dense,Input,concatenate
from keras.layers import BatchNormalization
from keras.models import Model
from keras.activations import relu
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.advanced_activations import ELU
from keras.applications.resnet50 import preprocess_input, resnet50
from math import sqrt
from keras import backend as K
from keras.callbacks import History 
import gc

Using TensorFlow backend.


In [2]:
%run '/home/santhosr/Documents/Codes/utils.py'

In [17]:
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input,rotation_range=50,horizontal_flip=True,validation_split=0.2)
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

In [18]:
batchSize = 15
imageSize = (512,512)

In [19]:
trainGen = train_datagen.flow_from_directory('/home/santhosr/Documents/Birad/ProcessedData/FullRes/',batch_size=batchSize,target_size=imageSize,subset='training',seed=123)
valGen = train_datagen.flow_from_directory('/home/santhosr/Documents/Birad/ProcessedData/FullRes/',batch_size=batchSize,target_size=imageSize,subset='validation',seed=123)


Found 4741 images belonging to 4 classes.
Found 1183 images belonging to 4 classes.


In [20]:
def conv_layer(feature_batch, feature_map, kernel_size=(3, 3),strides=(1,1), zp_flag=False):
    if zp_flag:
        zp = ZeroPadding2D((1,1))(feature_batch)
    else:
        zp = feature_batch
    conv = Conv2D(filters=feature_map, kernel_size=kernel_size, strides=strides)(zp)
#     bn = BatchNormalization(axis=3)(conv)
#     act = LeakyReLU(1/10)(bn)
    act = LeakyReLU(1/10)(conv)
    return act

In [21]:
inp = Input(shape=(imageSize[0], imageSize[1],3))

conv1 = conv_layer(inp, 64, kernel_size=(3,3), strides=(1,1),zp_flag=False)
conv2 = conv_layer(conv1, 64, kernel_size=(3,3), strides=(1,1), zp_flag=False)
mp1 = MaxPooling2D(pool_size=(3, 3), strides=(2, 2))(conv2)
# 23
conv3 = conv_layer(mp1, 128, zp_flag=False)
conv4 = conv_layer(conv3, 128, zp_flag=False)
conv4_2 = conv_layer(conv4, 128, zp_flag=False)
mp2 = MaxPooling2D(pool_size=(3, 3), strides=(2, 2))(conv4)
9
conv7 = conv_layer(mp2, 256, zp_flag=False)
conv8 = conv_layer(conv7, 256, zp_flag=False)
conv9 = conv_layer(conv8, 256, zp_flag=False)
mp3 = MaxPooling2D(pool_size=(3, 3), strides=(2, 2))(conv9)

conv10 = conv_layer(mp3, 256, zp_flag=False)
conv11 = conv_layer(conv10, 256, zp_flag=False)
mp4 = MaxPooling2D(pool_size=(3, 3), strides=(2, 2))(conv11)

conv12 = conv_layer(mp4, 512, strides=(2,2),zp_flag=False)
# conv13 = conv_layer(conv12, 256, zp_flag=False)
# mp5 = MaxPooling2D(pool_size=(3, 3), strides=(2, 2))(conv13)



# dense layers
# flt = Flatten()(conv12)
out1 = GlobalAvgPool2D()(conv12)
out2 = GlobalMaxPool2D()(conv12)
out = concatenate([out1,out2])
ds1 = Dense(256, activation='relu')(out)
dp1 = Dropout(0.4)(ds1)
# ds2 = Dense(64, activation='relu')(ds1)
out = Dense(4, activation='softmax')(dp1)

model = Model(inp, out)

In [9]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 512, 512, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 510, 510, 64) 1792        input_1[0][0]                    
__________________________________________________________________________________________________
leaky_re_lu_1 (LeakyReLU)       (None, 510, 510, 64) 0           conv2d_1[0][0]                   
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 508, 508, 64) 36928       leaky_re_lu_1[0][0]              
__________________________________________________________________________________________________
leaky_re_l

In [22]:
adam = Adam(lr=0.001)
model.compile(loss='categorical_crossentropy',optimizer = adam,metrics=['accuracy',f1])

In [9]:
def step_decay(epoch):
  initial_lrate = 0.000005
  drop = 0.4
  epochs_drop = 2.0
  lrate = initial_lrate * math.pow(drop,  
           math.floor((1+epoch)/epochs_drop))
  return lrate

lrate = LearningRateScheduler(step_decay)

In [23]:
model.fit_generator(trainGen,epochs=6,verbose=1,steps_per_epoch=len(trainGen),validation_data=valGen,validation_steps=len(valGen))

Epoch 1/6
317/317 [==============================] - 499s 2s/step - loss: 1.0813 - acc: 0.5714 - f1: 0.4497 - val_loss: 1.0364 - val_acc: 0.5824 - val_f1: 0.5824
Epoch 2/6
317/317 [==============================] - 488s 2s/step - loss: 1.0444 - acc: 0.5825 - f1: 0.5021 - val_loss: 1.0329 - val_acc: 0.5824 - val_f1: 0.5824
Epoch 3/6
317/317 [==============================] - 478s 2s/step - loss: 1.0389 - acc: 0.5796 - f1: 0.5446 - val_loss: 1.0319 - val_acc: 0.5824 - val_f1: 0.5824
Epoch 4/6
317/317 [==============================] - 498s 2s/step - loss: 1.0438 - acc: 0.5798 - f1: 0.5465 - val_loss: 1.0294 - val_acc: 0.5824 - val_f1: 0.5824
Epoch 5/6
317/317 [==============================] - 468s 1s/step - loss: 1.0345 - acc: 0.5825 - f1: 0.5724 - val_loss: 1.0283 - val_acc: 0.5824 - val_f1: 0.5824
Epoch 6/6
317/317 [==============================] - 468s 1s/step - loss: 1.0323 - acc: 0.5825 - f1: 0.5766 - val_loss: 1.0255 - val_acc: 0.5824 - val_f1: 0.5824


In [12]:
len(trainGen)

192